# Custom LLLM Agent

This notebook goes through how to create your own custom LLM agent.

An LLM agent consists of three parts:
    
    - Tools: The tools the agent has available to use.
    - LLMChain: The LLMChain that produces the text that is parsed in a certain way to determine which action to take.
    - OutputParser: This determines how to parse the LLMOutput into 
        
        
In this notebook we walk through how to create a custom LLM agent.

In [1]:
import sys
sys.path.append("../../../../../lang-chat/backend/")

In [20]:
from app.toolkits.open_api.toolkit import OpenAPIToolkit
urls = [
    "https://www.klarna.com/.well-known/ai-plugin.json"
]
toolkits = list(map(OpenAPIToolkit.from_plugin_url, urls))
namespaces = "\n\n".join([tk.get_typescript_namespace() for tk in toolkits])
#print(namespaces)

In [ ]:
from langchain.agents import Tool, AgentExecutor
from langchain.agents.agent import LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain

In [48]:
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

If you want use a tool you should write valid TypeScript code to call it. Please put all parameters and their values on their own line. If you use strings, they should be wrapped in double quotes not single.

For your response, you have two options:

If you do not need to use any tools, you can just respond directly to the user by using the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Final Answer: your response to user here

If you do need to use tools, you should use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: ```typescript
...
``` 
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question.

When repsonding in your final answer to the user, they have NO knowledge of any intermediate steps. So if there is any intermediate knowledge there you want them to know, you should make sure to return it as part of your final answer.
Begin!

Question: {input}
Thought: {agent_scratchpad}"""

In [49]:
class CustomPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = shopify.get_typescript_namespace()
        return template.format(**kwargs)

In [50]:
prompt = CustomPromptTemplate(input_variables=["input", "intermediate_steps"])

In [51]:
llm = OpenAI(temperature=0, max_tokens=-1)

In [52]:
query = "what are some cheap pokemon t-shirts?"

In [53]:
FINAL_ANSWER_ACTION = "Final Answer:"
import json
import json5
from langchain.schema import AgentAction, AgentFinish
import re
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output):
        if FINAL_ANSWER_ACTION in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split(FINAL_ANSWER_ACTION)[-1].strip()},
                log=llm_output,
            )
        parsed = llm_output.split("Action:")[1].split("```typescript")[1].split("```")[0].strip()
        typescript_str = parsed.split("(")[1][:-1]
        tool_input = json.dumps(json5.loads(typescript_str))
        return AgentAction(tool=parsed.split("(")[0], tool_input=tool_input, log=llm_output)

In [54]:
output_parser = CustomOutputParser()

In [55]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [56]:
agent = LLMSingleActionAgent(llm_chain=llm_chain, output_parser=output_parser, stop=["\nObservation:"])

In [57]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=shopify.get_tools(), verbose=True)

In [58]:
agent_executor.run(query)



> Entering new AgentExecutor chain...
 I need to find some product suggestions that match the query
Action: ```typescript
KlarnaProducts.productsUsingGET({
	q: "pokemon t-shirts",
	size: 10,
	budget: 20
})
```

Observation:{"products":[{"name":"Airwaves Men's Pokemon T-shirt","url":"https://www.klarna.com/us/shopping/pl/cl10001/3201808037/Clothing/Airwaves-Men-s-Pokemon-T-shirt/?utm_source=openai","price":"$17.49","attributes":["Material:Cotton","Target Group:Man","Color:Gray,Blue,Black"]},{"name":"Airwaves Men's Pokemon Diamond Shape Pikachu T-shirt","url":"https://www.klarna.com/us/shopping/pl/cl10001/3201810636/Clothing/Airwaves-Men-s-Pokemon-Diamond-Shape-Pikachu-T-shirt/?utm_source=openai","price":"$17.49","attributes":["Material:Cotton","Target Group:Man","Color:Gray,Black"]},{"name":"Airwaves Men's Pokemon Pikachu Lightening T-shirt","url":"https://www.klarna.com/us/shopping/pl/cl10001/3201823213/Clothing/Airwaves-Men-s-Pokemon-Pikachu-Lightening-T-shirt/?utm_source=openai","p

"Here are some cheap pokemon t-shirts I found for you: Airwaves Men's Pokemon T-shirt, Airwaves Men's Pokemon Diamond Shape Pikachu T-shirt, Airwaves Men's Pokemon Pikachu Lightening T-shirt, Fifth Sun Boy's Pokemon Get Ready to Battle Pikachu Retro Child T-Shirt Black, Pokemon Girl Eevee Face Graphic Tee Athletic Heather, Pokémon Kid's Piplup T-Shirt - White, Airwaves Men's Pokemon Retro T-shirt, Airwaves Pokemon Pikachu T-shirt - Black, Airwaves Men's Pokemon Pikachu T-shirt, and Pokemon Gengar Youth Short Sleeve Graphic T-Shirt. All of these t-shirts are under $20."

In [59]:
agent_executor.run("hi")



> Entering new AgentExecutor chain...
 Greeting the user
Final Answer: Hi there! How can I help you?

> Finished chain.


'Hi there! How can I help you?'